In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1b4.dev7
cdv, version 1.1.2
Python 3.10.7


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)
# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../code")
from utils import (load_program, print_program, print_puzzle, print_json)
import singleton_utils
import tic_tac_toe

# puzzles
tic_tac_toe_puzzle = load_program("../code/tic-tac-toe.clsp", ["../code"])
coin_puzzle = load_program("../code/coin.clsp", ["../code"])
terminate_puzzle = load_program("../code/terminate-game.clsp", ["../code"])

waiting_room_puzzle = load_program("../code/waiting-room.clsp", ["../code"])
play_amount = 1_000_000_000_000
game_amount = (play_amount * 2) + 1 # singleton odd mojos

In [3]:
# from setup-sim notebook
# alice 4070016934
# bob 861183802
alice_pk = G1Element.from_bytes(bytes.fromhex("96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010"))
alice_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice_pk).get_tree_hash()
bob_pk = G1Element.from_bytes(bytes.fromhex("b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a"))
bob_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(bob_pk).get_tree_hash()

# prepare players info
player_one_info = Program.to([alice_pk, alice_puzzle_hash])
player_two_info = Program.to([bob_pk, bob_puzzle_hash])


## Create Waiting Room Coins
[waiting-room.clsp](../code/waiting-room.clsp)
```clojure
(mod (
        PLAYER_PH
        PLAYER_PK 
        P1_PK
        P2_PK 
        RETURN_AMOUNT
        GAME_AMOUNT
        launcher_id
        singleton_full_puzzle_hash
    )
```

In [4]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    alice_puzzle_hash, 
    alice_pk,
    alice_pk, 
    bob_pk,
    play_amount,
    game_amount
)
alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()
alice_waiting_room_address = encode_puzzle_hash(alice_waiting_room_puzzle_hash, "txch")
alice_waiting_room_address

'txch10cy0fk9zs4slnlxvnq79gjdw8yfapdc82c7a0k6alljmrlyhcenqwwvzrt'

In [5]:
## bob
bob_waiting_room_puzzle = waiting_room_puzzle.curry(
    bob_puzzle_hash, 
    bob_pk,
    alice_pk, 
    bob_pk,
    play_amount,
    game_amount
    
)
bob_waiting_room_puzzle_hash = bob_waiting_room_puzzle.get_tree_hash()
bob_waiting_room_address = encode_puzzle_hash(bob_waiting_room_puzzle_hash, "txch")
bob_waiting_room_address

'txch1ar2fme6md4wysj87ygxy0dhahp7ym3eww4ejtjfm0f5wre99mx6qu7quv6'

In [6]:
%%bash
chia wallet send -f 4070016934 -t txch10cy0fk9zs4slnlxvnq79gjdw8yfapdc82c7a0k6alljmrlyhcenqwwvzrt -a 1.000000000001

Submitting transaction...
Transaction submitted to nodes: [{'peer_id': '38401aad891c6ecd90a91150989b51e9c86d844e07387ed77338beda85b199e7', 'inclusion_status': 'SUCCESS', 'error_msg': None}]
Run 'chia wallet get_transaction -f 4070016934 -tx 0x8baf4d6fca16b8daa1531e09d0eb7cdcf9bcfe71d786ae3aed26815b2bbf543f' to get status


In [7]:
%%bash
chia wallet send -f 861183802 -t txch1ar2fme6md4wysj87ygxy0dhahp7ym3eww4ejtjfm0f5wre99mx6qu7quv6 -a 1

Submitting transaction...
Transaction submitted to nodes: [{'peer_id': '38401aad891c6ecd90a91150989b51e9c86d844e07387ed77338beda85b199e7', 'inclusion_status': 'SUCCESS', 'error_msg': None}]
Run 'chia wallet get_transaction -f 861183802 -tx 0x1f175eb18a8061d7a32c622ee4a99270f5c09d950c380908fd7559240024926a' to get status


In [8]:
%%bash
cdv sim -n tic-tac-toe farm

Farmed 1 Transaction blocks
Block Height is now: 6


In [9]:
print(alice_waiting_room_puzzle_hash)
print(bob_waiting_room_puzzle_hash)

7e08f4d8a28561f9fccc983c5449ae3913d0b707563dd7db5dffe5b1fc97c666
e8d49de75b6d5c4848fe220c47b6fdb87c4dc72e757325c93b7a68e1e4a5d9b4


In [10]:
%%bash
cdv rpc coinrecords --by puzzlehash "7e08f4d8a28561f9fccc983c5449ae3913d0b707563dd7db5dffe5b1fc97c666"
cdv rpc coinrecords --by puzzlehash "e8d49de75b6d5c4848fe220c47b6fdb87c4dc72e757325c93b7a68e1e4a5d9b4"

[
    {
        "coin": {
            "amount": 1000000000001,
            "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
            "puzzle_hash": "0x7e08f4d8a28561f9fccc983c5449ae3913d0b707563dd7db5dffe5b1fc97c666"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1665217886
    }
]
[
    {
        "coin": {
            "amount": 1000000000000,
            "parent_coin_info": "0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97",
            "puzzle_hash": "0xe8d49de75b6d5c4848fe220c47b6fdb87c4dc72e757325c93b7a68e1e4a5d9b4"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1665217886
    }
]


In [ ]:
%%bash # reset to block 6
CUR_HEIGHT=$(chia rpc full_node get_blockchain_state | jq ".blockchain_state.peak.height")
echo $CUR_HEIGHT
echo $((CUR_HEIGHT-6))
cdv sim -n tic-tac-toe revert -fd --blocks $((CUR_HEIGHT-6))

## Terminate Puzzle (Curried)
[terminate-game.clsp](../code/terminate-game.clsp)
```clojure
(mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
```

In [ ]:
player_amount = 1_000_000_000_000
player_fee = 50_000_000
p2_amount = player_amount - player_fee
game_amount = (player_amount * 2) + 1 # odd amount for singleton
curried_terminate_puzzle = terminate_puzzle.curry(
    1, # Terminate Singleton Coin
    alice_puzzle_hash,
    bob_puzzle_hash,
    p2_amount
)

## Coin Puzzle (Curried)
[coin.clsp](../code/coin.clsp)
```clojure
(mod (MOD TERMINATE_PUZZLE PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE AMOUNT position)
```

In [ ]:
def get_coin_puzzle(board, player):
    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        curried_terminate_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle,
        game_amount)
    return curried_coin_puzzle

curried_coin_puzzle = get_coin_puzzle([' '] * 9, 'x')

## Launcher Coin

In [ ]:
from chia.rpc.full_node_rpc_client import FullNodeRpcClient
from chia.util.config import load_config
from chia.util.default_root import DEFAULT_ROOT_PATH
from chia.util.ints import uint16

config = load_config(DEFAULT_ROOT_PATH, "config.yaml")
selected_network = config["selected_network"]
genesis_challenge = config["farmer"]["network_overrides"]["constants"][selected_network]["GENESIS_CHALLENGE"]

self_hostname = config["self_hostname"] # localhost
full_node_rpc_port = config["full_node"]["rpc_port"] # 8555
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

alice_waiting_room_puzzle_coin = coin_records[0].coin

In [ ]:
from chia.types.blockchain_format.coin import Coin
from chia.types.coin_spend import CoinSpend
from chia.wallet.puzzles import singleton_top_layer_v1_1

# prepare launcher coin
launcher_coin = Coin(
    alice_waiting_room_puzzle_coin.name(), # waiting_room's coin spend creates the launcher coin
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH, 
    game_amount
)

launcher_id = launcher_coin.name()
print(f'\nlauncher id: {launcher_id}')

singleton_struct = (
    singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
    (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
)

singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
    singleton_struct,
    curried_coin_puzzle, # tic tac toe coin puzzle
)
launcher_coin

In [ ]:
launcher_solution = Program.to(
    [
        singleton_puzzle.get_tree_hash(),
        game_amount,
        [
            ("game", "tic tac toe"), 
            ("player one", alice_puzzle_hash),
            ("player two", bob_puzzle_hash),
        ]
    ]
)
launcher_announcement = launcher_solution.get_tree_hash()

launcher_coin_spend = CoinSpend(
    launcher_coin,
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER,
    launcher_solution
)
print_json(launcher_coin_spend.to_json_dict())